In [1]:
import pandas as pd
import os
import re
import numpy as np
from functools import reduce
import copy
import panel as pn
import statistics
from bs4 import BeautifulSoup
from pathlib import Path
import os.path
import glob
import csv

In [2]:
inputDir = "/home/mlk442/FIND/MicroMGx_Pilot/genomics/antismash_outputs"
strain = "DA976Z1625"
path_gcf = "/home/mlk442/FIND/mlk442/gcfs_data.txt"
outdir = "/home/mlk442/FIND/mlk442/BGC_table"

In [3]:
def main_function(strain,inputDir = "/home/mlk442/FIND/MicroMGx_Pilot/genomics/antismash_outputs", 
         path_gcf = "/home/mlk442/FIND/mlk442/gcfs_data.txt",
         outdir = "/home/mlk442/FIND/mlk442/BGC_table"):
    os.chdir(inputDir)
#os.chdir("C:\\Users\\mlk442\\Desktop\\manuscripts\\BGC_table\\")
#strain = "DA561A0323" 
#outdir = "C:\\Users\\mlk442\\Desktop\\manuscripts\\BGC_table\\"
    csv_path =outdir +"/" +strain+"_BGC_table.csv"
    with open (csv_path, "w") as f:
        csc_write = csv.writer(f)
        title ="BCGs in "+strain
        csv_head = ["BCGs in "+strain]
        csc_write.writerow(csv_head)

    gbkfiles = []
    for file in glob.glob(strain+"/*.gbk"):
        gbkfiles.append(file)
    N_gbk = len(gbkfiles) -1
    N_gbk      
#Read html file
    path = strain +"/index.html"
    htmlfile = open(path, "r", encoding = "utf-8")

#Creat gcfs(key) to gc(value) dictionary, which is specific to the strain
#Some gcs belong to more than one gcf
    gcfsData = pd.read_table(path_gcf, sep = "\t")
    dict_gcf = dict(zip(gcfsData.gcf.tolist(), gcfsData.bgcs.tolist()))
    dict_gc = dict()
    for key in dict_gcf.keys():
        gc_list = re.split(" ",dict_gcf[key])    
        filtered_list = list(filter(lambda x: re.match(strain, x) != None, gc_list))
        if len(filtered_list) >0 :
            for gc in filtered_list:
                if gc in dict_gc.keys():
                    dict_gc[gc].append(key)
                else:
                    dict_gc[gc] = [key]    
        
 #Process html file   
    htmlhandle = htmlfile.read()
    soup = BeautifulSoup(htmlhandle, "lxml")
    count = 0

#region(key) to genbank(value) dictionary
#warning information dictionary with region as key
    dict_genbank = dict()
    dict_warn = dict()
    genbank = soup.find_all("div")
    for i in range(len(genbank)):
        if ("Download region GenBank file" in genbank[i].get_text()):
            for link in genbank[i].find_all('a'):
                genbank_info = genbank[i].find_all('a')
                if len(genbank_info) >2:
                    name_genbank = genbank_info[1].get("href")
                    region_info2 = re.split("#|r|c",genbank_info[2].get("href"))
                    name_region2 = "Region "+str(int(region_info2[2]))+"."+region_info2[3]
                    dict_genbank[name_region2] = name_genbank 
            if ("Region on contig edge." in genbank[i].get_text()):
                dict_warn[name_region2] = 0
            
#Create dataframe                
    df = pd.DataFrame(columns = ["Region","Type", "From", "To", "Contig Edge Warning","GenBank", "ID", "GCFs"])            
    items = soup.find_all("tr")
    faile=0
    successed = 0
    j=0
    for i in range(len(items)):
    # Extract information from first 4 columns by "Region&nbsp" 
        if "Region&nbsp" in items[i].get_text(): 
            j += 1
            info = re.split("\n",items[i].get_text())
            Region = info[2].replace("&nbsp"," ")
            Type = info[5]
            From = info[7]
            To = info[8]
        #Find correaponding waring information in the warn dictionary
            if(Region in dict_warn.keys()):
                edgeWarn = "Region on contig edge"
            else:
                edgeWarn = ""
        #Creat GeneBank and ID information
            forGen = re.split(" |\.", Region)
            ID = strain +"_"+str(j)
        # Check whether Region exists or not, if not, add a row 
            if Region not in df.index.tolist():
            #Transform gdfs list to string
                if (ID in dict_gc.keys()):
                    GCFs = ", ".join(dict_gc[ID])
                else:
                    GCFs = ""
                GeneBank = strain +"/"+dict_genbank[Region]
            # Check if this Genbank file is in the antimash folder or not            
                if Path(GeneBank).is_file():
                    df.loc[Region] = [Region,Type,From,To, edgeWarn, GeneBank, ID, GCFs]
                    successed += 1
                else:
                    faile += 0                 
#df.loc["BGCs"] = "BCGs in "+strain
#print (failed)
    if (successed == N_gbk and faile ==0):
        print (strain +": completed successfully!")
    else: 
        print (strain + ": "+str(N_gbk) + "gbk file in the folder, "+ str(success)+" gbk file in the table, " 
               + str(failed)+ " GenBank files can't be find in the directory!")    
    df.to_csv(outdir +"/"+strain+"_BGC_table.csv", mode = "a", index = 0)
    return (df)

In [4]:
main_function("DA936Z0011").style.hide_index()

DA936Z0011: completed successfully!


Region,Type,From,To,Contig Edge Warning,GenBank,ID,GCFs
Region 1.1,lassopeptide,"33,562","56,385",,DA936Z0011/c00001_NODE_1_...region001.gbk,DA936Z0011_1,RiPPs_87
Region 2.1,NRPS-like,"126,630","149,087",Region on contig edge,DA936Z0011/c00002_NODE_2_...region001.gbk,DA936Z0011_2,NRPS_4403
Region 6.1,"NRPS,T1PKS",1,"31,082",Region on contig edge,DA936Z0011/c00006_NODE_6_...region001.gbk,DA936Z0011_3,"NRPS_338, PKS-NRP_Hybrids_17, PKSI_277"
Region 6.2,phosphonate,"70,166","94,223",Region on contig edge,DA936Z0011/c00006_NODE_6_...region002.gbk,DA936Z0011_4,Others_510
Region 9.1,butyrolactone,"1,269","12,195",,DA936Z0011/c00009_NODE_9_...region001.gbk,DA936Z0011_5,Others_26
Region 9.2,LAP,"64,472","82,925",Region on contig edge,DA936Z0011/c00009_NODE_9_...region002.gbk,DA936Z0011_6,RiPPs_2864
Region 11.1,siderophore,"16,016","27,794",,DA936Z0011/c00011_NODE_11...region001.gbk,DA936Z0011_7,Others_34
Region 13.1,NRPS,1,"43,311",Region on contig edge,DA936Z0011/c00013_NODE_13...region001.gbk,DA936Z0011_8,NRPS_669
Region 14.1,NRPS-like,"20,932","64,081",,DA936Z0011/c00014_NODE_14...region001.gbk,DA936Z0011_9,NRPS_4321
Region 19.1,terpene,1,"13,734",Region on contig edge,DA936Z0011/c00019_NODE_19...region001.gbk,DA936Z0011_10,Terpene_5


In [5]:
main_function("DA976Z1732").style.hide_index()

DA976Z1732: completed successfully!


Region,Type,From,To,Contig Edge Warning,GenBank,ID,GCFs
Region 1.1,terpene,"31,900","70,673",,DA976Z1732/CONTIG.region001.gbk,DA976Z1732_1,Terpene_658
Region 1.2,"NRPS,arylpolyene","236,259","302,077",,DA976Z1732/CONTIG.region002.gbk,DA976Z1732_2,"NRPS_779, Others_569"
Region 1.3,"T2PKS,NRPS","315,692","421,489",,DA976Z1732/CONTIG.region003.gbk,DA976Z1732_3,"NRPS_1308, PKS-NRP_Hybrids_351, PKSother_424"
Region 1.4,betalactone,"430,435","455,724",,DA976Z1732/CONTIG.region004.gbk,DA976Z1732_4,Others_1196
Region 2.1,NRPS,"29,933","92,354",,DA976Z1732/CONTIG_0.region001.gbk,DA976Z1732_5,NRPS_904
Region 2.2,"T1PKS,butyrolactone","110,001","224,918",,DA976Z1732/CONTIG_0.region002.gbk,DA976Z1732_6,"Others_621, PKSI_625"
Region 2.3,NRPS-like,"257,151","301,041",,DA976Z1732/CONTIG_0.region003.gbk,DA976Z1732_7,NRPS_1817
Region 3.1,NRPS-like,"179,329","221,868",,DA976Z1732/CONTIG_1.region001.gbk,DA976Z1732_8,NRPS_2045
Region 3.2,NRPS,"302,672","359,884",,DA976Z1732/CONTIG_1.region002.gbk,DA976Z1732_9,NRPS_121
Region 3.3,NRPS,"375,003","417,483",,DA976Z1732/CONTIG_1.region003.gbk,DA976Z1732_10,NRPS_432
